In [10]:
import pandas as pd
from numpy import *
import statsmodels.api as sm
import matplotlib.pylab as plt
import plotly.graph_objs as go
import plotly as py
import cufflinks as cf
import datetime as dt
import seaborn as sns
from sklearn import linear_model
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import scipy
import os
import pywt
import sqlite3
from sqlite3 import Error
import warnings
import itertools
from dateutil.relativedelta import *

warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

# matplotlib.rcParams['axes.labelsize'] = 14
# matplotlib.rcParams['xtick.labelsize'] = 12
# matplotlib.rcParams['ytick.labelsize'] = 12
# matplotlib.rcParams['text.color'] = 'k'

# py.tools.set_credentials_file(username='lhatpku', api_key='IQ9S1tothSzT9TmjMwnX')

maturities = asarray([1,2,3,6,12,24,36,60,84,120,240,360])
beta_names = ['beta1', 'beta2', 'beta3']
lam_t = .0609
_load2 = lambda x: (1.-exp(-lam_t*x)) / (lam_t*x)
_load3 = lambda x: ((1.-exp(-lam_t*x)) / (lam_t*x)) - exp(-lam_t*x)

In [14]:

def loadData():

    lam_t = .0609
    # filter where we only get the last day of every month

    loc ='treasury_yield.db'

    conn = sqlite3.connect(loc)
    ratedata = pd.read_sql('select * from treasury_yield', conn)

    lam_t = .0609
    _load2 = lambda x: (1.-exp(-lam_t*x)) / (lam_t*x)
    _load3 = lambda x: ((1.-exp(-lam_t*x)) / (lam_t*x)) - exp(-lam_t*x)
    ratedata = ratedata.drop('id', axis=1)

    ratedata = ratedata.set_index('Date')

    # make 2D matrix of all of the beta coeff. for each maturity
    X = zeros((len(maturities), 2))
    X[:,0] = _load2(maturities)
    X[:,1] = _load3(maturities)
    X = sm.add_constant(X)

    # for each observation, fit the maturity curve
    beta_fits = zeros((len(ratedata), 3))
    residuals = zeros((len(ratedata), len(maturities)))

    for i in range(0, len(ratedata)):

        ratedata_index = (ratedata.iloc[i].isnull()==0)
        ratedata_select = ratedata.iloc[i][ratedata_index]
        X = zeros((len(ratedata_select), 2))
        X[:,0] = _load2(maturities)[ratedata_index]
        X[:,1] = _load3(maturities)[ratedata_index]
        X = sm.add_constant(X)
        model = sm.regression.linear_model.OLS(ratedata_select, X)
        results = model.fit()
        beta_fits[i,:3] = results.params
        residuals[i,:][ratedata_index] = results.resid

    # convert into a dataframe for conv.
    beta_fits = pd.DataFrame(beta_fits, columns=beta_names)
    residuals = pd.DataFrame(residuals, columns=[str(mat) for mat in maturities])

    beta_fits.index = ratedata.index
    residuals.index = ratedata.index

    return beta_fits, residuals, ratedata

In [15]:
beta_fits, residuals, ratedata = loadData()
beta_fits_plot = pd.concat([beta_fits, ratedata], axis=1, join_axes=[beta_fits.index])
beta_fits_plot_reset = beta_fits_plot.reset_index()


In [17]:
records=beta_fits_plot_reset.to_dict('records')

records

[{'Date': '1990-01-02 00:00:00',
  'beta1': 8.013963163038444,
  'beta2': -0.1589095595152883,
  'beta3': -0.2389839987145166,
  'MO_1': nan,
  'MO_2': nan,
  'MO_3': 7.83,
  'MO_6': 7.89,
  'YR_1': 7.81,
  'YR_2': 7.87,
  'YR_3': 7.9,
  'YR_5': 7.87,
  'YR_7': 7.98,
  'YR_10': 7.94,
  'YR_20': nan,
  'YR_30': 8.0},
 {'Date': '1990-01-03 00:00:00',
  'beta1': 8.055647436911622,
  'beta2': -0.1538023542643039,
  'beta3': -0.18795038412632437,
  'MO_1': nan,
  'MO_2': nan,
  'MO_3': 7.89,
  'MO_6': 7.94,
  'YR_1': 7.85,
  'YR_2': 7.94,
  'YR_3': 7.96,
  'YR_5': 7.92,
  'YR_7': 8.04,
  'YR_10': 7.99,
  'YR_20': nan,
  'YR_30': 8.04},
 {'Date': '1990-01-04 00:00:00',
  'beta1': 8.052515528900514,
  'beta2': -0.19991994707719973,
  'beta3': -0.18536001436389427,
  'MO_1': nan,
  'MO_2': nan,
  'MO_3': 7.84,
  'MO_6': 7.9,
  'YR_1': 7.82,
  'YR_2': 7.92,
  'YR_3': 7.93,
  'YR_5': 7.91,
  'YR_7': 8.02,
  'YR_10': 7.98,
  'YR_20': nan,
  'YR_30': 8.04},
 {'Date': '1990-01-05 00:00:00',
  'beta

In [96]:
lam_t = .0609
maturities_output = list(range(3,363,6))

yield_rates=[]
maturity=[]
date=[]

for item in records:
    for y in maturities_output:
        load2 = (1. - math.exp(-lam_t*y)) / (lam_t*y);
        load3 = ((1.- math.exp(-lam_t*y)) / (lam_t*y)) - math.exp(-lam_t*y);

        yield_rate = item['beta1'] + item['beta2'] * load2 + item['beta3'] * load3;

        yield_rates.append(yield_rate)
        maturity.append(y)
        date.append(item['Date'])

In [103]:
df=pd.DataFrame({'yield_rates':yield_rates, 'maturity': maturity, 'Date':pd.to_datetime(date)})

df['Date']=df['Date'].dt.to_period('Y')
df.head()

,yield_rates,maturity,Date
0,7.849379,3,1990
1,7.845790,9,1990
2,7.848969,15,1990
3,7.855960,21,1990
4,7.864877,27,1990


In [111]:
gb=df.groupby(['maturity','Date']).mean()
df1=gb.reset_index()

df1

,maturity,Date,yield_rates
0,3,1990,7.744611
1,3,1991,5.495126
2,3,1992,3.454173
3,3,1993,3.084947
4,3,1994,4.476846
5,3,1995,5.732231
6,3,1996,5.185226
7,3,1997,5.248374
8,3,1998,4.995242
9,3,1999,4.823436


In [112]:
conn = sqlite3.connect('smooth_returns.db')
df1.to_sql(name='smooth_returns', con=conn, if_exists='replace')

InterfaceError: Error binding parameter 2 - probably unsupported type.

In [114]:
df1.describe()

,maturity,yield_rates
count,1800.000000,1800.000000
mean,180.000000,4.560879
std,103.937489,1.973891
min,3.000000,-0.064531
25%,91.500000,2.874563
50%,180.000000,4.651726
75%,268.500000,6.083037
max,357.000000,8.634586


In [76]:
df1=df['yield_rates'].resample('MS').mean()

In [77]:
df1

1990-01-01    8.201520
1990-02-01    8.459288
1990-03-01    8.573671
1990-04-01    8.744826
1990-05-01    8.716946
1990-06-01    8.451722
1990-07-01    8.436400
1990-08-01    8.689021
1990-09-01    8.820633
1990-10-01    8.642132
1990-11-01    8.333015
1990-12-01    8.029526
1991-01-01    8.014842
1991-02-01    7.776217
1991-03-01    8.026203
1991-04-01    7.941432
1991-05-01    7.965941
1991-06-01    8.179209
1991-07-01    8.161611
1991-08-01    7.797881
1991-09-01    7.565934
1991-10-01    7.439232
1991-11-01    7.319056
1991-12-01    7.001152
1992-01-01    6.941503
1992-02-01    7.218319
1992-03-01    7.426038
1992-04-01    7.354587
1992-05-01    7.269959
1992-06-01    7.160946
                ...   
2016-10-01    1.864615
2016-11-01    2.209328
2016-12-01    2.512330
2017-01-01    2.446042
2017-02-01    2.447033
2017-03-01    2.522312
2017-04-01    2.363154
2017-05-01    2.380894
2017-06-01    2.265813
2017-07-01    2.372184
2017-08-01    2.278559
2017-09-01    2.272895
2017-10-01 